# Implicit feature selection

## Setup

In [53]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## Data

### Import data

In [54]:
df = pd.read_csv('https://raw.githubusercontent.com/kirenz/datasets/master/advertising.csv')

### Data structure

In [55]:
df

,Market,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Market     200 non-null    int64  
 1   TV         200 non-null    float64
 2   radio      200 non-null    float64
 3   newspaper  200 non-null    float64
 4   sales      200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


### Data corrections

In [57]:
# variable Market is categorical
df['Market'] = df['Market'].astype('category')

### Variable lists

In [58]:
# define outcome variable as y_label
y_label = 'sales'

# select features
features = df.drop(columns=[y_label, 'Market']).columns

# create feature data
X = df[features]

# create response
y = df[y_label]

### Data splitting

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [60]:
# data exploration set
df_train = pd.DataFrame(X_train.copy())
df_train = df_train.join(pd.DataFrame(y_train))

### Analyze data

In [61]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,160.0,150.019375,84.418857,0.7,77.750,150.65,218.825,296.4
radio,160.0,22.875625,14.805216,0.0,9.825,21.20,36.425,49.6
newspaper,160.0,29.945625,20.336449,0.3,12.875,25.60,44.500,100.9
sales,160.0,14.100000,5.108754,1.6,10.475,13.20,17.325,27.0


In [62]:
alt.Chart(df_train).mark_bar().encode(
    alt.X(alt.repeat("column"), type="quantitative", bin=True),
    y='count()',
).properties(
    width=150,
    height=150
).repeat(
    column=['sales', 'TV', 'radio', 'newspaper']
)

alt.RepeatChart(...)

In [63]:
alt.Chart(df_train).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=150,
    height=150
).repeat(
    row=['sales', 'TV', 'radio', 'newspaper'],
    column=['sales', 'TV', 'radio', 'newspaper']
).interactive()

alt.RepeatChart(...)

In [64]:
# inspect correlation between outcome and possible predictors
corr = df_train.corr()
corr[y_label].sort_values(ascending=False)

sales        1.000000
TV           0.768874
radio        0.592373
newspaper    0.237874
Name: sales, dtype: float64

In [65]:
# take a look at all correlations
corr.style.background_gradient(cmap='Blues')

,TV,radio,newspaper,sales
TV,1.000000,0.053872,0.019084,0.768874
radio,0.053872,1.000000,0.388074,0.592373
newspaper,0.019084,0.388074,1.000000,0.237874
sales,0.768874,0.592373,0.237874,1.000000


## Lasso regression model

- Lasso regression relies upon the linear regression model but additionaly performs a so called L1 regularization, which is a process of introducing additional information in order to prevent overfitting. 

- As a consequence, we can fit a model containing all possible predictors and use lasso to perform variable selection by using a technique that regularizes the coefficient estimates (it shrinks the coefficient estimates towards zero). 

- Lasso performs best when all numerical features are centered around 0 and have variance in the same order.

- If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.

- This means it is important to standardize our features. We do this by subtracting the mean from our observations and then dividing the difference by the standard deviation.

### Prepare data

- To avoid data leakage, the standardization of numerical features should always be performed after data splitting and only from training data. 

- Furthermore, we obtain all necessary statistics for our features (mean and standard deviation) from training data and also use them on test data. 

- Note that we don’t standardize our dummy variables (which only have values of 0 or 1).

In [66]:
scaler = StandardScaler().fit(X_train[features]) 

X_train[features] = scaler.transform(X_train[features])
X_test[features] = scaler.transform(X_test[features])

In [67]:
X_train

,TV,radio,newspaper
79,-0.404248,-1.028237,-0.337675
197,0.320608,-0.919828,-1.161439
38,-1.270511,0.259124,0.254251
24,-1.042359,-0.696233,-0.574446
122,0.879103,-1.387343,-0.707629
...,...,...,...
106,-1.485591,-0.804643,-0.012116
14,0.642634,0.679210,0.791917
92,0.804241,0.719863,1.433170
179,0.185143,-0.872399,-0.608975


### Select model

In [68]:
# select the lasso model with built in crossvalidation
reg = LassoCV(cv=5, random_state=0)

### Training & best alpha

In [69]:
reg.fit(X_train, y_train)

LassoCV(cv=5, random_state=0)

Show best value of penalization chosen by cross validation:

In [70]:
reg.alpha_

0.05951504753162918

### Fit best model

In [71]:
# Fit the model to the complete training data
reg = Lasso(alpha=reg.alpha_)
reg.fit(X_train, y_train)

Lasso(alpha=0.05951504753162918)

### Coefficients

In [72]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": features,
    "Coefficient": reg.coef_}
)

# combine estimates of intercept and slopes
table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,14.100
1,TV,3.708
2,radio,2.753
3,newspaper,0.013


### Evaluation on test set

In [73]:
# obtain predictions
y_pred = reg.predict(X_test)

In [74]:
# R squared
r2_score(y_test, y_pred).round(3)

0.899

In [75]:
# MSE
mean_squared_error(y_test, y_pred).round(3)

3.182

In [76]:
# RMSE
mean_squared_error(y_test, y_pred, squared=False).round(3)

1.784

In [77]:
# MAE
mean_absolute_error(y_test, y_pred).round(3)

1.455